In [1]:
import polars as pl
import pandas as pd
import numpy as np

## Сравнение времени работы Pandas и Polars

In [2]:
n = 10**7
data = {
    'user_id': np.random.randint(1, 100000, n),
    'login_time': pd.date_range(start='2024-10-01', periods=n, freq='s'),
    'logout_time': pd.date_range(start='2024-10-01', periods=n, freq='s') + pd.to_timedelta(np.random.randint(0, 10000, n), unit='s'),
    'value': np.random.randn(n)
}

In [3]:
df_pandas = pd.DataFrame(data)
df_pandas.head(5)

,user_id,login_time,logout_time,value
0,43779,2024-10-01 00:00:00,2024-10-01 02:33:15,-1.677755
1,41814,2024-10-01 00:00:01,2024-10-01 01:52:33,0.700644
2,69023,2024-10-01 00:00:02,2024-10-01 01:31:06,1.105641
3,69030,2024-10-01 00:00:03,2024-10-01 01:34:33,-0.979762
4,34817,2024-10-01 00:00:04,2024-10-01 00:58:17,0.093287


In [4]:
df_polars = pl.DataFrame(data)
df_polars.head(5)

user_id,login_time,logout_time,value
i64,datetime[ns],datetime[ns],f64
43779,2024-10-01 00:00:00,2024-10-01 02:33:15,-1.677755
41814,2024-10-01 00:00:01,2024-10-01 01:52:33,0.700644
69023,2024-10-01 00:00:02,2024-10-01 01:31:06,1.105641
69030,2024-10-01 00:00:03,2024-10-01 01:34:33,-0.979762
34817,2024-10-01 00:00:04,2024-10-01 00:58:17,0.093287


### Pandas

In [5]:
%%time

df_pandas_filtered = df_pandas[df_pandas['value'] > 0.5]
agg_pandas = df_pandas_filtered.groupby('user_id').agg({'value': 'mean'}).reset_index()

CPU times: user 636 ms, sys: 636 ms, total: 1.27 s
Wall time: 1.39 s


### Polars

In [6]:
%%time

df_polars_filtered = df_polars.filter(pl.col('value') > 0.5)
agg_polars = df_polars_filtered.group_by('user_id').agg(pl.col('value').mean())

CPU times: user 702 ms, sys: 436 ms, total: 1.14 s
Wall time: 611 ms


## А теперь немного познакомимся с Polars

Загрузим датасет

Датасет взят с этого соревнования на Kaggle

https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store

In [5]:
df = pl.read_csv("kz.csv")

In [6]:
df.shape

(2633521, 8)

In [7]:
df.head(5)

event_time,order_id,product_id,category_id,category_code,brand,price,user_id
str,i64,i64,i64,str,str,f64,i64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434
"""2020-04-24 19:16:21 UTC""",2294584263154074236,2273948316817424439,2268105471367840086,null,"""karcher""",217.57,1515915625443148002


In [8]:
df.dtypes

[String, Int64, Int64, Int64, String, String, Float64, Int64]

> В Polars можно индексироваться, используя `[]`, но это является ***антипаттерном***

In [9]:
df[0,:]

event_time,order_id,product_id,category_id,category_code,brand,price,user_id
str,i64,i64,i64,str,str,f64,i64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984


In [10]:
df[:,['event_time', "price"]].head()

event_time,price
str,f64
"""2020-04-24 11:50:39 UTC""",162.01
"""2020-04-24 11:50:39 UTC""",162.01
"""2020-04-24 14:37:43 UTC""",77.52
"""2020-04-24 14:37:43 UTC""",77.52
"""2020-04-24 19:16:21 UTC""",217.57


In [39]:
df.row(1)

('2020-04-24 11:50:39 UTC',
 2294359932054536986,
 1515966223509089906,
 2268105426648170900,
 'electronics.tablet',
 'samsung',
 162.01,
 1515915625441993984)

In [17]:
# Работает, но так лучше не делать
# df.filter(df["price"] > 100)

# Лучше делать так 
df.filter(pl.col("price") > 1000).head()

event_time,order_id,product_id,category_id,category_code,brand,price,user_id
str,i64,i64,i64,str,str,f64,i64
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-29 17:52:37 UTC""",2298165996101304765,1515966223509089406,2268105428166508982,"""electronics.smartphone""","""apple""",1433.54,1515915625452182512


В Polars SQL-подобный синтаксис

In [18]:
df.select(['brand', 'price']).head()

brand,price
str,f64
"""samsung""",162.01
"""samsung""",162.01
"""huawei""",77.52
"""huawei""",77.52
"""karcher""",217.57


Сохраняет все оригинальные колонки и добавляет новую. Аналогично `SELECT *` в SQL

In [19]:
df.with_columns([
    (pl.col("price") * 100).alias("price_x_100")
]).head()

event_time,order_id,product_id,category_id,category_code,brand,price,user_id,price_x_100
str,i64,i64,i64,str,str,f64,i64,f64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,16201.0
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,16201.0
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,7752.0
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,7752.0
"""2020-04-24 19:16:21 UTC""",2294584263154074236,2273948316817424439,2268105471367840086,null,"""karcher""",217.57,1515915625443148002,21757.0


Вычисления для колонок, используя `select`

In [20]:
df.select([
    pl.col("product_id").n_unique()
])

product_id
u32
25113


In [21]:
df.filter(
    pl.col('brand').is_in(['apple', 'samsung', 'motorola'])
).head()

event_time,order_id,product_id,category_id,category_code,brand,price,user_id
str,i64,i64,i64,str,str,f64,i64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480


In [22]:
df.select([
    pl.col("price").min().alias("min"),
    pl.col("price").mean().alias("mean"),
    pl.col("price").median().alias("median"),
    pl.col("price").max().alias("max"),
    pl.col("price").std().alias("std_dev"),
])

min,mean,median,max,std_dev
f64,f64,f64,f64,f64
0.0,154.093165,55.53,50925.9,241.942109


In [23]:
df.select([
    pl.col("price")
]).describe()

statistic,price
str,f64
"""count""",2.201567e6
"""null_count""",431954.0
"""mean""",154.093165
"""std""",241.942109
"""min""",0.0
"""25%""",14.56
"""50%""",55.53
"""75%""",196.74
"""max""",50925.9


In [24]:
df_top = df.head()

In [25]:
df_top.with_columns([
    ("brand-" + pl.col("brand")).alias("brand2"),
    (pl.col("price") * 100).alias("price2")
])

event_time,order_id,product_id,category_id,category_code,brand,price,user_id,brand2,price2
str,i64,i64,i64,str,str,f64,i64,str,f64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,"""brand-samsung""",16201.0
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,"""brand-samsung""",16201.0
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,"""brand-huawei""",7752.0
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,"""brand-huawei""",7752.0
"""2020-04-24 19:16:21 UTC""",2294584263154074236,2273948316817424439,2268105471367840086,null,"""karcher""",217.57,1515915625443148002,"""brand-karcher""",21757.0


In [26]:
df.with_columns([
    pl.col("price").mean().over("category_code").alias("price_by_categorycode")
]).head(10)

event_time,order_id,product_id,category_id,category_code,brand,price,user_id,price_by_categorycode
str,i64,i64,i64,str,str,f64,i64,f64
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,275.879351
"""2020-04-24 11:50:39 UTC""",2294359932054536986,1515966223509089906,2268105426648170900,"""electronics.tablet""","""samsung""",162.01,1515915625441993984,275.879351
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,65.151228
"""2020-04-24 14:37:43 UTC""",2294444024058086220,2273948319057183658,2268105430162997728,"""electronics.audio.headphone""","""huawei""",77.52,1515915625447879434,65.151228
"""2020-04-24 19:16:21 UTC""",2294584263154074236,2273948316817424439,2268105471367840086,null,"""karcher""",217.57,1515915625443148002,59.252235
"""2020-04-26 08:45:57 UTC""",2295716521449619559,1515966223509261697,2268105442636858090,"""furniture.kitchen.table""","""maestro""",39.33,1515915625450382722,38.172499
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480,286.965165
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480,286.965165
"""2020-04-26 09:33:47 UTC""",2295740594749702229,1515966223509104892,2268105428166508982,"""electronics.smartphone""","""apple""",1387.01,1515915625448766480,286.965165


In [27]:
df.group_by("brand").agg([
    pl.len()
]).head()

brand,len
str,u32
"""1515915625513578007""",2
"""1515915625514719973""",2
"""1515915625484635847""",1
"""1515915625484572163""",1
"""1515915625469956347""",1


In [28]:
df.group_by("brand").agg([
    pl.len().alias('count')
]).sort("count", descending=True).head()

brand,count
str,u32
null,506005
"""samsung""",358928
"""ava""",118161
"""tefal""",78653
"""apple""",74709


In [29]:
df['brand'].value_counts()

brand,count
str,u32
"""1515915625492654241""",1
"""1515915625514593891""",1
"""1515915625512333600""",1
"""speedo""",4
"""1515915625484671477""",1
…,…
"""1515915625484665702""",1
"""1515915625484570845""",4
"""1515915625512084215""",2


In [30]:
df_brand_event = df.group_by(["brand", "category_code"]).agg([
    pl.len()
])

df_brand_event.head()

brand,category_code,len
str,str,u32
"""1515915625512764064""","""9.26""",1
"""continent""","""auto.accessories.compressor""",384
"""1515915625513444346""","""3.89""",1
"""1515915625512283849""","""8.10""",1
"""1515915625514802421""","""3.89""",1


## Дополнительные источники

1. [Официальная страница Polars](https://pola.rs)
2. [Getting started Polars](https://docs.pola.rs/user-guide/getting-started/#reading-writing)
3. [Polars vs. pandas: What’s the Difference?](https://blog.jetbrains.com/pycharm/2024/07/polars-vs-pandas/)